# 数据探索与清洗

因为按照常理助学金是给家庭有经济困难的学生的，而学生的消费情况比较能反映学生的经济状况，所以本次探索会先清洗出学生的各类消费情况及总消费。
学习成绩也应该会对奖学金的评定有一定的影响，以我在本科读书时的经验来看，成绩好的能拿奖学金一般不会申请或者不能通过助学金的审批，而曾经太差也会影响，所以我第二步准备将学生成绩清洗为在学院的排名，及学院人数这两个变量。

### 步骤
#### step1 清晰学生饭卡数据

#### step2 清洗学生学习成绩数据

## 处理学生消费数据

In [20]:
import pandas as pd 
import warnings 
warnings.filterwarnings("ignore")
names_card = [ '学生id','消费类别','消费地点','消费方式','消费时间','消费金额','剩余金额']
# 导入学生饭卡数据
card_train = pd.read_csv("../input/card_train.txt",header=None,encoding='utf-8',names = names_card,index_col = False)
card_test = pd.read_csv("../input/card_test.txt",header=None,encoding='utf-8',names = names_card,index_col = False)
card_data = pd.concat([card_train,card_test])

names_score = ['学生id','学院编号','成绩排名']
# 导入学习成绩数据
score_train = pd.read_csv("../input/score_train.txt",header=None,encoding='utf-8',names = names_score,index_col = False)
score_test = pd.read_csv("../input/score_test.txt",header=None,encoding='utf-8',names = names_score,index_col = False)
score_data = pd.concat([score_train,score_test])

# 删除后面不用的变量，节省内存
del card_train,card_test,score_train,score_test,names_card,names_score

In [2]:
print(card_data.shape)
print(score_data.shape)

(24848402, 7)
(18130, 3)


In [3]:
len(card_data.学生id.unique())

21631

In [4]:
# 处理消费方式变量
card_data['消费方式'] = card_data['消费方式'].astype('category')
card_data['消费方式'].describe()

count     23998315
unique          11
top             食堂
freq       9027893
Name: 消费方式, dtype: object

In [5]:
# 看看有多少种消费方式
print(card_data['消费方式'].unique())

[淋浴, 开水, 其他, 洗衣房, 文印中心, ..., 食堂, 校车, NaN, 超市, 校医院]
Length: 12
Categories (11, object): [淋浴, 开水, 其他, 洗衣房, ..., 食堂, 校车, 超市, 校医院]


数据总共记录了11类消费数据（不含Nan），23998315次消费，其中食堂消费次数最多

In [6]:
# 缺失值处理
# 查看Ｎａｎ
card_data.isnull().sum()

学生id         0
消费类别     62723
消费地点    850087
消费方式    850087
消费时间         0
消费金额         0
剩余金额         0
dtype: int64

In [7]:
# 由于消费方式中食堂消费最多，使用食堂代替缺失的消费类别
card_data['消费方式'].fillna('食堂',inplace=True)

In [8]:
#每个学生的总消费
card_sum_by_ID = card_data.groupby('学生id')['消费金额'].sum()

In [9]:
card_sum_by_ID.head(20)

学生id
0      4997.56
1      6182.69
2      4966.65
3        42.40
8     11472.37
9     14502.91
10     6028.74
11     5976.20
16     2974.56
17      495.85
19     3331.06
20     6393.73
21     3165.76
22    11328.89
23    10959.76
28     6401.58
29     3845.78
32     2093.69
33       97.52
34     7647.02
Name: 消费金额, dtype: float64

In [10]:
## 每个学生各类别的消费
card_sum_by_ID_type = card_data.groupby(['学生id','消费方式'])['消费金额'].sum().unstack('消费方式')

In [11]:
card_sum_by_ID_type.head()

消费方式,其他,图书馆,开水,教务处,文印中心,校医院,校车,洗衣房,淋浴,超市,食堂
学生id,,,,,,,,,,,
0,NaN,84.4,429.58,NaN,0.3,NaN,417.49,NaN,12.30,839.68,3213.81
1,NaN,324.2,265.97,29.3,96.4,NaN,119.40,25.02,22.50,514.30,4785.60
2,0.01,22.4,670.36,NaN,NaN,NaN,34.88,22.20,53.10,829.95,3333.75
3,NaN,NaN,0.18,NaN,NaN,NaN,NaN,7.62,34.60,NaN,NaN
8,NaN,425.9,2440.94,NaN,2.6,NaN,2211.45,4.50,39.83,10.00,6337.15


In [12]:
# 缺失值
card_sum_by_ID_type.fillna(0,inplace=True)

In [13]:
card_sum_by_ID_type.head()

消费方式,其他,图书馆,开水,教务处,文印中心,校医院,校车,洗衣房,淋浴,超市,食堂
学生id,,,,,,,,,,,
0,0.00,84.4,429.58,0.0,0.3,0.0,417.49,0.00,12.30,839.68,3213.81
1,0.00,324.2,265.97,29.3,96.4,0.0,119.40,25.02,22.50,514.30,4785.60
2,0.01,22.4,670.36,0.0,0.0,0.0,34.88,22.20,53.10,829.95,3333.75
3,0.00,0.0,0.18,0.0,0.0,0.0,0.00,7.62,34.60,0.00,0.00
8,0.00,425.9,2440.94,0.0,2.6,0.0,2211.45,4.50,39.83,10.00,6337.15


In [14]:
## 结合数据
card = pd.concat([card_sum_by_ID,card_sum_by_ID_type],axis =1)

In [16]:
card.head()

,消费金额,其他,图书馆,开水,教务处,文印中心,校医院,校车,洗衣房,淋浴,超市,食堂
学生id,,,,,,,,,,,,
0,4997.56,0.00,84.4,429.58,0.0,0.3,0.0,417.49,0.00,12.30,839.68,3213.81
1,6182.69,0.00,324.2,265.97,29.3,96.4,0.0,119.40,25.02,22.50,514.30,4785.60
2,4966.65,0.01,22.4,670.36,0.0,0.0,0.0,34.88,22.20,53.10,829.95,3333.75
3,42.40,0.00,0.0,0.18,0.0,0.0,0.0,0.00,7.62,34.60,0.00,0.00
8,11472.37,0.00,425.9,2440.94,0.0,2.6,0.0,2211.45,4.50,39.83,10.00,6337.15


In [17]:
del card_sum_by_ID,card_sum_by_ID_type,card_data


## 处理学生学习成绩数据

In [23]:
# 将成绩排名标准化
score_data.成绩排名= score_data.groupby('学院编号').成绩排名.transform(lambda x: (x - x.mean()) / x.std())

In [26]:
score_data.set_index('学生id')

,学院编号,成绩排名
学生id,,
0,9,-1.681326
1,9,-1.680156
8,6,1.749205
9,6,1.760045
10,3,-1.709162
11,3,-1.707670
19,7,-1.647174
20,7,-0.412101
21,7,-0.323628


In [34]:
# 保存数据
score_data.to_csv('../input/cleaned_score.csv')
card.to_csv('../input/cleaned_card.csv')

In [ ]:
# http://pandas.pydata.org/pandas-docs/stable/io.html#io-read-csv-table